# Pattern Analyzer

### Smart Connect Prerequisites

In [ ]:
import SmartApi
from six.moves.urllib.parse import urljoin
import json
import logging
import SmartApi.smartExceptions as ex
import requests
from requests import get
import re, uuid
import socket
import os
import logzero
from logzero import logger
import time

from SmartApi.version import __version__, __title__

log = logging.getLogger(__name__)

class SmartConnect(object):
    #_rootUrl = "https://openapisuat.angelbroking.com"
    _rootUrl="https://apiconnect.angelbroking.com" #prod endpoint
    #_login_url ="https://smartapi.angelbroking.com/login"
    _login_url="https://smartapi.angelbroking.com/publisher-login" #prod endpoint
    _default_timeout = 7  # In seconds

    _routes = {
        "api.login":"/rest/auth/angelbroking/user/v1/loginByPassword",
        "api.logout":"/rest/secure/angelbroking/user/v1/logout",
        "api.token": "/rest/auth/angelbroking/jwt/v1/generateTokens",
        "api.refresh": "/rest/auth/angelbroking/jwt/v1/generateTokens",
        "api.user.profile": "/rest/secure/angelbroking/user/v1/getProfile",

        "api.order.place": "/rest/secure/angelbroking/order/v1/placeOrder",
        "api.order.placefullresponse": "/rest/secure/angelbroking/order/v1/placeOrder",
        "api.order.modify": "/rest/secure/angelbroking/order/v1/modifyOrder",
        "api.order.cancel": "/rest/secure/angelbroking/order/v1/cancelOrder",
        "api.order.book":"/rest/secure/angelbroking/order/v1/getOrderBook",
        
        "api.ltp.data": "/rest/secure/angelbroking/order/v1/getLtpData",
        "api.trade.book": "/rest/secure/angelbroking/order/v1/getTradeBook",
        "api.rms.limit": "/rest/secure/angelbroking/user/v1/getRMS",
        "api.holding": "/rest/secure/angelbroking/portfolio/v1/getHolding",
        "api.position": "/rest/secure/angelbroking/order/v1/getPosition",
        "api.convert.position": "/rest/secure/angelbroking/order/v1/convertPosition",

        "api.gtt.create":"/gtt-service/rest/secure/angelbroking/gtt/v1/createRule",
        "api.gtt.modify":"/gtt-service/rest/secure/angelbroking/gtt/v1/modifyRule",
        "api.gtt.cancel":"/gtt-service/rest/secure/angelbroking/gtt/v1/cancelRule",
        "api.gtt.details":"/rest/secure/angelbroking/gtt/v1/ruleDetails",
        "api.gtt.list":"/rest/secure/angelbroking/gtt/v1/ruleList",

        "api.candle.data":"/rest/secure/angelbroking/historical/v1/getCandleData",
        "api.market.data":"/rest/secure/angelbroking/market/v1/quote",
        "api.search.scrip": "/rest/secure/angelbroking/order/v1/searchScrip",
        "api.allholding": "/rest/secure/angelbroking/portfolio/v1/getAllHolding",

        "api.individual.order.details": "/rest/secure/angelbroking/order/v1/details/",
        "api.margin.api" : 'rest/secure/angelbroking/margin/v1/batch'
    }

    try:
        clientPublicIp= " " + get('https://api.ipify.org').text
        if " " in clientPublicIp:
            clientPublicIp=clientPublicIp.replace(" ","")
        hostname = socket.gethostname()
        clientLocalIp=socket.gethostbyname(hostname)
    except Exception as e:
        logger.error(f"Exception while retriving IP Address,using local host IP address: {e}")
    finally:
        clientPublicIp="106.193.147.98"
        clientLocalIp="127.0.0.1"
    clientMacAddress=':'.join(re.findall('..', '%012x' % uuid.getnode()))
    accept = "application/json"
    userType = "USER"
    sourceID = "WEB"

    def __init__(self, api_key=None, access_token=None, refresh_token=None,feed_token=None, userId=None, root=None, debug=False, timeout=None, proxies=None, pool=None, disable_ssl=False,accept=None,userType=None,sourceID=None,Authorization=None,clientPublicIP=None,clientMacAddress=None,clientLocalIP=None,privateKey=None):
        self.debug = debug
        self.api_key = api_key
        self.session_expiry_hook = None
        self.disable_ssl = disable_ssl
        self.access_token = access_token
        self.refresh_token = refresh_token
        self.feed_token = feed_token
        self.userId = userId
        self.proxies = proxies if proxies else {}
        self.root = root or self._rootUrl
        self.timeout = timeout or self._default_timeout
        self.Authorization= None
        self.clientLocalIP=self.clientLocalIp
        self.clientPublicIP=self.clientPublicIp
        self.clientMacAddress=self.clientMacAddress
        self.privateKey=api_key
        self.accept=self.accept
        self.userType=self.userType
        self.sourceID=self.sourceID
        # Create a log folder based on the current date
        log_folder = time.strftime("%Y-%m-%d", time.localtime())
        log_folder_path = os.path.join("logs", log_folder)  # Construct the full path to the log folder
        os.makedirs(log_folder_path, exist_ok=True) # Create the log folder if it doesn't exist
        log_path = os.path.join(log_folder_path, "app.log") # Construct the full path to the log file
        logzero.logfile(log_path, loglevel=logging.INFO)  # Output logs to a date-wise log file

        if pool:
            self.reqsession = requests.Session()
            reqadapter = requests.adapters.HTTPAdapter(**pool)
            self.reqsession.mount("https://", reqadapter)
            logger.info(f"in pool")
        else:
            self.reqsession = requests

        # disable requests SSL warning
        requests.packages.urllib3.disable_warnings()
    def requestHeaders(self):
        return{
            "Content-type":self.accept,
            "X-ClientLocalIP": self.clientLocalIp,
            "X-ClientPublicIP": self.clientPublicIp,
            "X-MACAddress": self.clientMacAddress,
            "Accept": self.accept,
            "X-PrivateKey": self.privateKey,
            "X-UserType": self.userType,
            "X-SourceID": self.sourceID
        }

    def setSessionExpiryHook(self, method):
        if not callable(method):
            raise TypeError("Invalid input type. Only functions are accepted.")
        self.session_expiry_hook = method
    
    def getUserId():
        return userId

    def setUserId(self,id):
        self.userId=id

    def setAccessToken(self, access_token):

        self.access_token = access_token

    def setRefreshToken(self, refresh_token):

        self.refresh_token = refresh_token

    def setFeedToken(self,feedToken):
        
        self.feed_token=feedToken

    def getfeedToken(self):
        return self.feed_token

    
    def login_url(self):
        """Get the remote login url to which a user should be redirected to initiate the login flow."""
        return "%s?api_key=%s" % (self._login_url, self.api_key)
    
    def _request(self, route, method, parameters=None):
        """Make an HTTP request."""
        param = parameters.copy() if parameters else {}
       
        uri =self._routes[route].format(**param)
        url = urljoin(self.root, uri)


        # Custom headers
        headers = self.requestHeaders()

        if self.access_token:
            # set authorization header
        
            auth_header = self.access_token
            headers["Authorization"] = "Bearer {}".format(auth_header)

        if self.debug:
            log.debug("Request: {method} {url} {param} {headers}".format(method=method, url=url, param=param, headers=headers))
    
        try:
            r = requests.request(method,
                                        url,
                                        data=json.dumps(param) if method in ["POST", "PUT"] else None,
                                        param=json.dumps(param) if method in ["GET", "DELETE"] else None,
                                        headers=headers,
                                        verify=not self.disable_ssl,
                                        allow_redirects=True,
                                        timeout=self.timeout,
                                        proxies=self.proxies)
           
        except Exception as e:
            raise e

        if self.debug:
            log.debug("Response: {code} {content}".format(code=r.status_code, content=r.content))

        # Validate the content type.
        if "json" in headers["Content-type"]:
            try:
                data = json.loads(r.content.decode("utf8"))
             
            except ValueError:
                raise ex.DataException("Couldn't parse the JSON response received from the server: {content}".format(
                    content=r.content))

            # api error
            if data.get("error_type"):
                # Call session hook if its registered and TokenException is raised
                if self.session_expiry_hook and r.status_code == 403 and data["error_type"] == "TokenException":
                    self.session_expiry_hook()

                # native errors
                exp = getattr(ex, data["error_type"], ex.GeneralException)
                raise exp(data["message"], code=r.status_code)

            return data
        elif "csv" in headers["Content-type"]:
            return r.content
        else:
            raise ex.DataException("Unknown Content-type ({content_type}) with response: ({content})".format(
                content_type=headers["Content-type"],
                content=r.content))
        
    def _deleteRequest(self, route, param=None):
        """Alias for sending a DELETE request."""
        return self._request(route, "DELETE", param)
    def _putRequest(self, route, param=None):
        """Alias for sending a PUT request."""
        return self._request(route, "PUT", param)
    def _postRequest(self, route, param=None):
        """Alias for sending a POST request."""
        return self._request(route, "POST", param)
    def _getRequest(self, route, param=None):
        """Alias for sending a GET request."""
        return self._request(route, "GET", param)

    def generateSession(self,clientCode,password,totp):
        
        param={"clientcode":clientCode,"password":password,"totp":totp}
        loginResultObject=self._postRequest("api.login",param)
        
        if loginResultObject['status']==True:
            jwtToken=loginResultObject['data']['jwtToken']
            self.setAccessToken(jwtToken)
            refreshToken = loginResultObject['data']['refreshToken']
            feedToken = loginResultObject['data']['feedToken']
            self.setRefreshToken(refreshToken)
            self.setFeedToken(feedToken)
            user = self.getProfile(refreshToken)

            id = user['data']['clientcode']
            # id='D88311'
            self.setUserId(id)
            user['data']['jwtToken'] = "Bearer " + jwtToken
            user['data']['refreshToken'] = refreshToken
            user['data']['feedToken'] = feedToken

            return user
        else:
            return loginResultObject
            
    def terminateSession(self,clientCode):
        logoutResponseObject=self._postRequest("api.logout",{"clientcode":clientCode})
        return logoutResponseObject

    def generateToken(self,refresh_token):
        response=self._postRequest('api.token',{"refreshToken":refresh_token})
        jwtToken=response['data']['jwtToken']
        feedToken=response['data']['feedToken']
        self.setFeedToken(feedToken)
        self.setAccessToken(jwtToken)

        return response

    def renewAccessToken(self):
        response =self._postRequest('api.refresh', {
            "jwtToken": self.access_token,
            "refreshToken": self.refresh_token,
            
        })
       
        tokenSet={}

        if "jwtToken" in response:
            tokenSet['jwtToken']=response['data']['jwtToken']
        tokenSet['clientcode']=self. userId   
        tokenSet['refreshToken']=response['data']["refreshToken"]
       
        return tokenSet

    def getProfile(self,refreshToken):
        user=self._getRequest("api.user.profile",{"refreshToken":refreshToken})
        return user

    def placeOrder(self,orderparam):
        param=orderparam
        for k in list(param.keys()):
            if param[k] is None :
                del(param[k])
        response= self._postRequest("api.order.place", param)
        if response is not None and response.get('status', False):
            if 'data' in response and response['data'] is not None and 'orderid' in response['data']:
                orderResponse = response['data']['orderid']
                return orderResponse
            else:
                logger.error(f"Invalid response format: {response}")
        else:
            logger.error(f"API request failed: {response}")
        return None

    def placeOrderFullResponse(self,orderparam):
        param=orderparam
        for k in list(param.keys()):
            if param[k] is None :
                del(param[k])
        response= self._postRequest("api.order.placefullresponse", param)
        if response is not None and response.get('status', False):
            if 'data' in response and response['data'] is not None and 'orderid' in response['data']:
                orderResponse = response
                return orderResponse
            else:
                logger.error(f"Invalid response format: {response}")
        else:
            logger.error(f"API request failed: {response}")
        return None
    
    def modifyOrder(self,orderparam):
        param = orderparam

        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])

        orderResponse= self._postRequest("api.order.modify", param)
        return orderResponse
    
    def cancelOrder(self, order_id,variety):
        orderResponse= self._postRequest("api.order.cancel", {"variety": variety,"orderid": order_id})
        return orderResponse

    def ltpData(self,exchange,tradingsymbol,symboltoken):
        param={
            "exchange":exchange,
            "tradingsymbol":tradingsymbol,
            "symboltoken":symboltoken
        }
        ltpDataResponse= self._postRequest("api.ltp.data",param)
        return ltpDataResponse
    
    def orderBook(self):
        orderBookResponse=self._getRequest("api.order.book")
        return orderBookResponse
        

    def tradeBook(self):
        tradeBookResponse=self._getRequest("api.trade.book")
        return tradeBookResponse
    
    def rmsLimit(self):
        rmsLimitResponse= self._getRequest("api.rms.limit")
        return rmsLimitResponse
    
    def position(self):
        positionResponse= self._getRequest("api.position")
        return positionResponse

    def holding(self):
        holdingResponse= self._getRequest("api.holding")
        return holdingResponse
    
    def allholding(self):
        allholdingResponse= self._getRequest("api.allholding")
        return allholdingResponse
    
    def convertPosition(self,positionparam):
        param=positionparam
        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])
        convertPositionResponse= self._postRequest("api.convert.position",param)

        return convertPositionResponse

    def gttCreateRule(self,createRuleparam):
        param=createRuleparam
        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])

        createGttRuleResponse=self._postRequest("api.gtt.create",param)
        return createGttRuleResponse['data']['id']

    def gttModifyRule(self,modifyRuleparam):
        param=modifyRuleparam
        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])
        modifyGttRuleResponse=self._postRequest("api.gtt.modify",param)
        return modifyGttRuleResponse['data']['id']
     
    def gttCancelRule(self,gttCancelparam):
        param=gttCancelparam
        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])
        cancelGttRuleResponse=self._postRequest("api.gtt.cancel",param)
        return cancelGttRuleResponse
     
    def gttDetails(self,id):
        param={
            "id":id
            }
        gttDetailsResponse=self._postRequest("api.gtt.details",param)
        return gttDetailsResponse
    
    def gttLists(self,status,page,count):
        if type(status)== list:
            param={
                "status":status,
                "page":page,
                "count":count
            }
            gttListResponse=self._postRequest("api.gtt.list",param)
            return gttListResponse
        else:
            message="The status param is entered as" +str(type(status))+". Please enter status param as a list i.e., status=['CANCELLED']"
            return message

    def getCandleData(self,historicParam):
        param=historicParam
        for k in list(param.keys()):
            if param[k] is None:
                del(param[k])
        getCandleDataResponse=self._postRequest("api.candle.data",historicParam)
        return getCandleDataResponse
    
    def getMarketData(self,mode,exchangeTokens):
        param={
            "mode":mode,
            "exchangeTokens":exchangeTokens
        }
        marketDataResult=self._postRequest("api.market.data",param)
        return marketDataResult
    
    def searchScrip(self, exchange, searchscrip):
        param = {
            "exchange": exchange,
            "searchscrip": searchscrip
        }
        searchScripResult = self._postRequest("api.search.scrip", param)
        if searchScripResult["status"] is True and searchScripResult["data"]:
            message = f"Search successful. Found {len(searchScripResult['data'])} trading symbols for the given query:"
            symbols = ""
            for index, item in enumerate(searchScripResult["data"], start=1):
                symbol_info = f"{index}. exchange: {item['exchange']}, tradingsymbol: {item['tradingsymbol']}, symboltoken: {item['symboltoken']}"
                symbols += "\n" + symbol_info
            logger.info(message + symbols)
            return searchScripResult
        elif searchScripResult["status"] is True and not searchScripResult["data"]:
            logger.info("Search successful. No matching trading symbols found for the given query.")
            return searchScripResult
        else:
            return searchScripResult
        
    def make_authenticated_get_request(self, url, access_token):
        headers = self.requestHeaders()
        if access_token:
            headers["Authorization"] = "Bearer " + access_token
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = json.loads(response.text)
            return data
        else:
            logger.error(f"Error in make_authenticated_get_request: {response.status_code}")
            return None
            
    def individual_order_details(self, qParam):
        url = self._rootUrl + self._routes["api.individual.order.details"] + qParam
        try:
            response_data = self.make_authenticated_get_request(url, self.access_token)
            return response_data
        except Exception as e:
            logger.error(f"Error occurred in ind_order_details: {e}")
            return None
    
    def getMarginApi(self,param):
        marginApiResult=self._postRequest("api.margin.api",param)
        return marginApiResult
 
    def _user_agent(self):
        return (__title__ + "-python/").capitalize() + __version__   



In [ ]:
from SmartApi import SmartConnect 
import pyotp
from logzero import logger
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import pandas as pd 
import requests
from datetime import datetime, timedelta
import credentials
import requests
import numpy as np
from time import time, sleep
import ta
import threading
import warnings
warnings.filterwarnings('ignore')

 ### Login

#### Login Info

In [ ]:
# -------------  login info ----------------
api_key = 'Enter Your API Key'
CLIENT_ID = username =  'Enter your CLIENT ID'
CLIENT_PIN = pwd = 'Enter Login PIN'
TOTP_CODE = token = "Generate and enter TOPT token"



#### Authentication

In [ ]:
# ---------- User Authentication ----------
import SmartApi

obj=SmartConnect(api_key=api_key)
data=obj.generateSession(username,pwd,pyotp.TOTP(token).now())
#print(data)
data_df = pd.DataFrame(data)
print(data_df)
AUTH_TOKEN=data['data']['jwtToken']
refreshToken=data['data']['refreshToken']
FEED_TOKEN=obj.generateToken(refreshToken)
res=obj.getProfile(refreshToken)


#### Fund Balance

In [ ]:
#-------------------- fund balance --------------------
import pandas as pd 
funds=obj.rmsLimit()
funds_df = pd.DataFrame(funds)
funds_df.head()


### Websocket

#### Method 1 

In [ ]:

#-------------------- Websocket Code-----------------
correlation_id = "abc123"
action = 1
mode = 1

token_list = [
    {
        "exchangeType": 1,
        "tokens": ["26009"]
    }
]
token_list1 = [
    {
        "action": 0,
        "exchangeType": 1,
        "tokens": ["26009"]
    }
]

sws = SmartWebSocketV2(AUTH_TOKEN, api_key, username, FEED_TOKEN)

def on_data(wsapp, message):
    logger.info("Ticks: {}".format(message))
    # close_connection()

def on_open(wsapp):
    logger.info("on open")
    sws.subscribe(correlation_id, mode, token_list)
    # sws.unsubscribe(correlation_id, mode, token_list1)


def on_error(wsapp, error):
    logger.error(error)


def on_close(wsapp):
    logger.info("Close")



def close_connection():
    sws.close_connection()


#- Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error
sws.on_close = on_close

sws.connect()

In [ ]:
#----------- Websocket code ----------------
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import threading, time
correlation_id = "abc123"
action = 1
mode = 1

token_list = [
    {
        "exchangeType": 1,
        "tokens": ["26009"]
    }
]


sws = SmartWebSocketV2(AUTH_TOKEN, api_key,username, FEED_TOKEN)

def on_data(wsapp, message):
    logger.info("Ticks: {}".format(message))
    # close_connection()

def on_open(wsapp):
    logger.info("on open")
    sws.subscribe(correlation_id, mode, token_list)
    print(f'Subscribe {token_list}')


def on_error(wsapp, error):
    logger.error(error)


def on_close(wsapp):
    logger.info("Close")



def close_connection():
    sws.close_connection()
    print('connection closed')


#- Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error
sws.on_close = on_close

sws.connect()

threading.Thread(target=sws.connect).start()
print('Control is released')
time.sleep(10)  


In [ ]:
"""import datetime 
import numpy as np 
import pandas as pd 
import requests , time
url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
token_df = pd.DataFrame.from_dict(requests.get(url).json())
token_df = token_df.astype({'strike': float})
token_df.to_csv(r'Enter CSV file path',header=True , index=False)"""

### Historical Data 

#### Historical API Function

In [ ]:
import pandas as pd 
from datetime import datetime
import time
import json
def historicalData():
    try:
        historicParam={
        "exchange": "NSE",
        "symboltoken": 1333,
        "interval": "FIFTEEN_MINUTE",
        "fromdate": "2024-02-20 09:15", 
        "todate": "2024-02-27 15:30"
        }
        return obj.getCandleData(historicParam)
    except Exception as e:
        print("Historic Api failed: {}".format(e.message))

res_json = historicalData()
columns = ['timestamp','open','high','low','close','volume']
stock_data1 = pd.DataFrame(res_json['data'],columns=columns)
stock_data1['timestamp'] = pd.to_datetime(stock_data1['timestamp'],format = "mixed")
stock_data1['timestamp'] = stock_data1['timestamp'].dt.tz_localize(None)

stock_data1.to_csv(r'Enter your file path', index=False)

time.sleep(1)

stock_data1.head()

In [ ]:
import ta
from ta import add_all_ta_features
from ta.utils import dropna
stock_data1 = dropna(stock_data1)

stock_data1.head()


### Adding indicators

#### SMA EMA MACD


In [ ]:
import pandas as pd 
from ta.trend import sma_indicator
stock_data1['SMA_100'] = ta.trend.sma_indicator(stock_data1['close'],100, False)
stock_data1['EMA_50'] = ta.trend.ema_indicator(stock_data1['close'],50,False)
stock_data1['MACD'] = ta.trend.macd(stock_data1['close'],12,26,False)
stock_data1['MACD_signal'] = ta.trend.macd_signal(stock_data1['close'],12,26,9,False)
stock_data1['MACD_diff'] = ta.trend.macd_diff(stock_data1['close'],12,26,9,False)
stock_data1.tail()

#### EMA

#### MACD

In [ ]:
import pandas as pd
import numpy as np
def HA(stock_data1):
    stock_data1['HA_close']=(stock_data1['open']+ stock_data1['high']+ stock_data1['low']+ stock_data1['close'])/4
    stock_data1['HA_open']=(stock_data1['open']+stock_data1['close'])/2   
    
    for i in range(1, len(stock_data1)):
        stock_data1['HA_open'][i]=(stock_data1['HA_open'][i-1]+stock_data1['HA_close'][i-1])/2 
    stock_data1['HA_high']=stock_data1[['HA_open','HA_close','high']].max(axis=1)
    stock_data1['HA_low']=stock_data1[['HA_open','HA_close','low']].min(axis=1)
    stock_data1 = pd.concat[['HA_close']]
    
    return stock_data1
#stock_data1['HA_Open'] = stock_data1['HA']

stock_data1.tail()

In [ ]:
import pandas as pd 
from ta.trend import sma_indicator
stock_data1['HA'] = ta.trend.sma_indicator(stock_data1['close'],100, False)

In [ ]:
stock_data1.tail()

In [ ]:
import cufflinks as cf
import plotly 
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
df1 = pd.read_csv("data6.csv")

In [ ]:

df1['timestamp'] = pd.to_datetime(df1['timestamp'])

def plot_time_series(df, column):
    plt.figure(figsize=(12, 5))
    plt.plot(df['timestamp'], df[column], label=column, color='b')
    plt.xlabel('Time')
    plt.ylabel(column.capitalize())
    plt.title(f'Time Series Plot of {column.capitalize()}')
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.show()

def plot_moving_average(df, column, window=5):
    df['MA'] = df[column].rolling(window=window).mean()
    plt.figure(figsize=(12, 5))
    plt.plot(df['timestamp'], df[column], label=f'{column.capitalize()} Price', alpha=0.6)
    plt.plot(df['timestamp'], df['MA'], label=f'{window}-Period Moving Avg', color='r', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel(column.capitalize())
    plt.title(f'Moving Average ({window}-Period) of {column.capitalize()}')
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.show()

def plot_correlation_heatmap(df):
    plt.figure(figsize=(8, 6))
    sns.heatmap(df.drop(columns=['timestamp']).corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Heatmap')
    plt.show()

def plot_distribution(df, column):
    plt.figure(figsize=(8, 5))
    sns.histplot(df[column], bins=30, kde=True, color='g')
    plt.xlabel(column.capitalize())
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {column.capitalize()}')
    plt.grid()
    plt.show()

plot_time_series(df1, 'close')
plot_moving_average(df1, 'close', window=10)
plot_correlation_heatmap(df1)
plot_distribution(df1, 'volume')

def plot_candlestick(df):
    fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                                         open=df['open'],
                                         high=df['high'],
                                         low=df['low'],
                                         close=df['close'])])
    fig.update_layout(title="Candlestick Chart", xaxis_title="Time", yaxis_title="Price")
    fig.show()

def plot_volume_bar(df):
    plt.figure(figsize=(12, 5))
    plt.bar(df['timestamp'], df['volume'], color='purple', alpha=0.6)
    plt.xlabel("Time")
    plt.ylabel("Volume")
    plt.title("Trading Volume Over Time")
    plt.xticks(rotation=45)
    plt.grid()
    plt.show()

def plot_daily_returns(df):
    df['daily_return'] = df['close'].pct_change() * 100  # Convert to percentage
    plt.figure(figsize=(8, 5))
    sns.histplot(df['daily_return'].dropna(), bins=30, kde=True, color='blue')
    plt.xlabel("Daily Return (%)")
    plt.ylabel("Frequency")
    plt.title("Histogram of Daily Returns")
    plt.grid()
    plt.show()
def plot_volatility(df, window=10):
    df['volatility'] = df['close'].rolling(window=window).std()
    plt.figure(figsize=(12, 5))
    plt.plot(df['timestamp'], df['volatility'], label=f'{window}-Period Volatility', color='red')
    plt.xlabel("Time")
    plt.ylabel("Volatility (Rolling Std Dev)")
    plt.title(f"Volatility Over Time ({window}-Period Rolling Std)")
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.show()
def plot_pairplot(df):
    sns.pairplot(df.drop(columns=['timestamp']).dropna())
    plt.show()

def plot_boxplot(df):
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df[['open', 'high', 'low', 'close']], palette="Set2")
    plt.title("Box Plot of Stock Prices")
    plt.ylabel("Price")
    plt.grid()
    plt.show()

plot_candlestick(df1)
plot_volume_bar(df1)
plot_daily_returns(df1)
plot_volatility(df1, window=10)
plot_pairplot(df1)
plot_boxplot(df1)
